<a href="https://colab.research.google.com/github/whoami-Lory271/DL-project/blob/main/transformer_base.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install library import 

In [ ]:
!pip install pytorch-lightning
!import pytorch_lightning as pl

In [ ]:
from numpy import array
from sklearn.preprocessing import OneHotEncoder
import math
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.modules.normalization import LayerNorm

from numpy import array
from sklearn.preprocessing import OneHotEncoder
import math
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.modules.normalization import LayerNorm
import pickle
import os
from torch.utils.data import Dataset, DataLoader
from torchvision.transforms import ToTensor
from pathlib import Path
import pandas as pd
import numpy as np
import csv

#Preprocessing

In [ ]:
save_path = "/content/drive/MyDrive/Deep learning" #change if needed

In [ ]:
#take all the txt files
dataset_path="/content/drive/MyDrive/Deep learning/train_data"
ft_texts_path=[str(path) for path in Path(dataset_path).glob("*.txt")]
print(ft_texts_path)

['/content/drive/MyDrive/Deep learning/train_data/probability__swr_p_level_set_easy.txt', '/content/drive/MyDrive/Deep learning/train_data/algebra__linear_1d_easy.txt', '/content/drive/MyDrive/Deep learning/train_data/calculus__differentiate_easy.txt', '/content/drive/MyDrive/Deep learning/train_data/calculus__differentiate_hard.txt', '/content/drive/MyDrive/Deep learning/train_data/probability__swr_p_level_set_hard.txt', '/content/drive/MyDrive/Deep learning/train_data/algebra__linear_1d_hard.txt', '/content/drive/MyDrive/Deep learning/train_data/probability__swr_p_level_set_medium.txt', '/content/drive/MyDrive/Deep learning/train_data/algebra__linear_1d_medium.txt', '/content/drive/MyDrive/Deep learning/train_data/calculus__differentiate_medium.txt']


In [ ]:
def create_vocabulary_from_directory(sentences_path):
  vocabulary = {}
  vocabulary['<sos>'] = 1
  vocabulary['<eos>'] = 2
  vocabulary['<pad>'] = 0
  index = 3
  conta = 0
  for file in sentences_path:
    with open(file, 'r') as file_in:
      text = file_in.read()
      sentences = text.split('\n')
      sentences_new = sentences[:-1] #remove last element because it's empty
      print("num sentences", len(sentences_new))
      for s in sentences_new:
        conta +=1
        for t in s:
         if t not in vocabulary:
            vocabulary[t] = index
            index += 1
      file_in.close()
  return vocabulary, conta

In [ ]:
vocabulary, conta = create_vocabulary_from_directory(ft_texts_path)
print(conta)

num sentences 1333332
num sentences 1333332
num sentences 1333332
num sentences 1333332
num sentences 1333332
num sentences 1333332
num sentences 1333332
num sentences 1333332
num sentences 1333332
11999988


In [ ]:
#create the pickle
outfile = open(os.path.join(save_path,"vocabs"), 'wb')
pickle.dump(vocabulary, outfile)
outfile.close()

In [ ]:
def encoding_from_sentence(vocab, sentence):
  enc= [1]
  for token in sentence:
    enc.append(vocab[token])
  enc.append(2)
  return enc

In [ ]:
def create_padding(encode, vocab, max_len):
    enc = np.array(encode)
    encoded = np.zeros(max_len + 2)
    for k in range(len(enc)):
      encoded[k] = enc[k]
    return encoded

In [ ]:
def create_pickles(vocabulary, sentences_paths, column_names=["Sentence", "Encoding"], path_quest="/content/drive/MyDrive/Deep learning/questions", path_answ="/content/drive/MyDrive/Deep learning/answers"):
  quest = []
  answ = []
  out_file_quest = open(path_quest, 'wb')
  out_file_answ = open(path_answ, 'wb')
  for file in sentences_paths:
    with open(file, 'r') as file_in:
      text = file_in.read()
      sentences = text.split('\n')
      sentences = sentences[:-1]
      print("num sentences", len(sentences))
      for i, sentence in enumerate(sentences):
          enc = encoding_from_sentence(vocabulary, sentence)
          if i % 2 == 0: #question
            quest.append({"Sentence": sentence, "Encoding": enc})
          elif i % 2 == 1: #answer
            answ.append({"Sentence": sentence, "Encoding": enc})
  pickle.dump(quest, out_file_quest)
  pickle.dump(answ, out_file_answ)
  out_file_quest.close()
  out_file_answ.close()
  return path_quest, path_answ

##Dataset

In [ ]:
#unpickle the vocabulary
infile = open(os.path.join(save_path,'vocabs'),'rb')
vocab = pickle.load(infile)
print(vocab)

{'<sos>': 1, '<eos>': 2, '<pad>': 0, 'W': 3, 'h': 4, 'a': 5, 't': 6, ' ': 7, 'i': 8, 's': 9, 'p': 10, 'r': 11, 'o': 12, 'b': 13, 'f': 14, 'c': 15, 'k': 16, 'n': 17, 'g': 18, '1': 19, 'd': 20, 'w': 21, 'e': 22, 'l': 23, 'u': 24, 'm': 25, '?': 26, '4': 27, '/': 28, '9': 29, 'y': 30, '{': 31, ':': 32, ',': 33, '2': 34, 'z': 35, '}': 36, '5': 37, 'F': 38, '6': 39, '.': 40, '3': 41, '8': 42, 'j': 43, '0': 44, 'C': 45, 'T': 46, 'x': 47, '7': 48, 'v': 49, 'q': 50, 'G': 51, 'S': 52, '=': 53, '*': 54, '+': 55, '-': 56, 'D': 57}


In [ ]:
path_quest, path_answ = create_pickles(vocab, ft_texts_path)

num sentences 1333332
num sentences 1333332
num sentences 1333332
num sentences 1333332
num sentences 1333332
num sentences 1333332
num sentences 1333332
num sentences 1333332
num sentences 1333332


In [ ]:
#unpickle the qestions
infile = open(os.path.join(save_path,"questions"),'rb')
quest = pickle.load(infile)
print(len(quest))

#unpickle the answers
infile = open(os.path.join(save_path,"answers"),'rb')
answ = pickle.load(infile)
print(len(answ))

5999994
5999994


In [ ]:
print(answ[0])

{'Sentence': '4/9', 'Encoding': [1, 27, 28, 29, 2]}


In [ ]:
class Custom_Dataset(Dataset):
  def __init__(self, file_quest, file_answ, vocab , max_len_quest=160, max_len_answ=30):
    self.file_quest = file_quest
    self.file_answ = file_answ
    self.vocab = vocab
    self.max_len_quest = max_len_quest
    self.max_len_answ = max_len_answ

  def __len__(self):
    return len(self.file_quest)

  def __getitem__(self, idx):
    assert(idx  < len(self.file_quest)) #indices should start from 0 to len - 1 (there are 5999994 elements)
    question = self.file_quest[idx]["Encoding"]
    encoding1 = torch.from_numpy(create_padding(question, self.vocab, self.max_len_quest))
    encoding1 = encoding1.type(torch.int)
    #self.sample["Question"] = encoding
      
    answer = self.file_answ[idx]["Encoding"]
    encoding2 = torch.from_numpy(create_padding(answer, self.vocab, self.max_len_answ))
    encoding2 = encoding2.type(torch.int)
    #self.sample["Answer"] = encoding
      
    return encoding1, encoding2 

In [ ]:
#Create dataset
dataset = Custom_Dataset(quest, answ, vocab)

In [ ]:
quest = None
answ = None

In [ ]:
print(dataset[0])

(tensor([ 1,  3,  4,  5,  6,  7,  8,  9,  7, 10, 11, 12, 13,  7, 12, 14,  7, 10,
         8, 15, 16,  8, 17, 18,  7, 19,  7, 13,  7,  5, 17, 20,  7, 19,  7, 10,
         7, 21,  4, 22, 17,  7,  6, 21, 12,  7, 23, 22,  6,  6, 22, 11,  9,  7,
        10,  8, 15, 16, 22, 20,  7, 21,  8,  6,  4, 12, 24,  6,  7, 11, 22, 10,
        23,  5, 15, 22, 25, 22, 17,  6,  7, 14, 11, 12, 25,  7,  6, 10, 10, 10,
        13, 13, 10, 13, 13, 13, 26,  2,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       dtype=torch.int32), tensor([ 1, 27, 28, 29,  2,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       dtype=torch.int32))


In [ ]:
#Create dataloader
train_loader = DataLoader(dataset, 32, shuffle=True)
print(len(train_loader))

187500


In [ ]:
print("data", next(iter(train_loader)))
print("questions_shape ", next(iter(train_loader))[0].shape)

print("answers_shape ", next(iter(train_loader))[1].shape)

data [tensor([[ 1, 38,  8,  ...,  0,  0,  0],
        [ 1, 38,  8,  ...,  0,  0,  0],
        [ 1, 52, 12,  ...,  0,  0,  0],
        ...,
        [ 1, 52, 12,  ...,  0,  0,  0],
        [ 1, 46, 21,  ...,  0,  0,  0],
        [ 1, 52, 12,  ...,  0,  0,  0]], dtype=torch.int32), tensor([[ 1, 34, 54,  ...,  0,  0,  0],
        [ 1, 56, 37,  ...,  0,  0,  0],
        [ 1, 19, 39,  ...,  0,  0,  0],
        ...,
        [ 1, 34, 39,  ...,  0,  0,  0],
        [ 1, 19, 37,  ...,  0,  0,  0],
        [ 1, 56, 34,  ...,  0,  0,  0]], dtype=torch.int32)]
questions_shape  torch.Size([32, 162])
answers_shape  torch.Size([32, 32])


# Model

In [ ]:
def create_vocabulary(sentences):
  vocabulary = {}
  vocabulary['<sos>'] = 1
  vocabulary['<eos>'] = 2
  vocabulary['<pad>'] = 0
  index = 3
  for s in sentences:
    tokens = s.split()
    for t in tokens:
      if t not in vocabulary:
        vocabulary[t] = index
        index += 1
  return vocabulary

In [ ]:
def encoding(vocabulary,sentences):
  X = [[vocabulary[token]] for sentence in sentences for token in sentence.split()]
  X = torch.tensor(X)
  X = X.reshape((len(sentences),X.shape[0] // len(sentences)))
  
  pad = X == vocabulary['<pad>']
  padding_mask = pad.repeat(1,1,X.shape[1]).reshape((X.shape[0],X.shape[1],X.shape[1]))
  padding_mask[pad] = True

  return X,padding_mask

In [ ]:
class EmbeddingLayer(nn.Module):
  def __init__(self,vocabulary_size,embedding_dim):
    super().__init__()
    self.E = nn.Embedding(vocabulary_size,embedding_dim)
  
  def forward(self,x):
    return self.E(x)

In [ ]:
def position_embedding(batch_size,seq_length,emb_dim,pad_mask):
  res = torch.zeros((batch_size,seq_length,emb_dim),dtype=torch.float32)
  for pos in range(seq_length):
    for i in range(emb_dim):
      if i%2 == 0:
        res[:,pos,i] = math.sin(pos/10000**(2*i/emb_dim))
      else:
        res[:,pos,i] = math.cos(pos/10000**(2*i/emb_dim))
  res[pad_mask[:,0,:]] = 0
  return res

In [ ]:
class TPSelfAttention(nn.Module):
  def __init__(self,dz,dk):
    super().__init__()
    self.dk = dk
    self.Wq = nn.Linear(dz,dk)
    self.Wk = nn.Linear(dz,dk)
    self.Wv = nn.Linear(dz,dk)
    self.Wr = nn.Linear(dz,dk)
    self.Wo = nn.Linear(dk,dz)
    self.softmax = nn.Softmax(dim=2)
  
  def forward(self,x,padding_mask,enc=None,mask=False,other_mask=None):
    q = self.Wq(x)
    r = self.Wr(x)
    if enc == None:
      k = self.Wk(x)
      v = self.Wv(x)
    else:
      k = self.Wk(enc)
      v = self.Wv(enc)
    sc = torch.matmul(q,k.permute(0,2,1)) / math.sqrt(self.dk)

    if other_mask == None:
      sc[padding_mask] = float('-inf')
    else:
      qmod = q.clone()
      kmod = k.clone()
      qmod[padding_mask[:,0,:]] = 0
      kmod[other_mask[:,0,:]] = 0
      sc = torch.matmul(qmod,kmod.permute(0,2,1)) / math.sqrt(self.dk)
      sc[sc == 0] = float('-inf')

    if mask==True:
      for i in range(sc.shape[1]):
        sc[:,i,i+1:] = float('-inf')
    score = torch.matmul(torch.nan_to_num(self.softmax(sc)),v)
    return self.Wo(r * score)

In [ ]:
class TPMultiHeadAttention(nn.Module):
  def __init__(self,dmodel,nhead,dropout=0.1):
    super().__init__()
    self.nhead = nhead
    self.att_layers = nn.ModuleList([TPSelfAttention(dmodel,dmodel // nhead) for i in range(nhead)])
    self.drop = nn.Dropout(p=dropout)
  
  def forward(self,x,padding_mask,enc=None,mask=False,other_mask=None):
    y = self.att_layers[0](x,padding_mask,enc=enc,mask=mask,other_mask=other_mask)
    for i in range(1,self.nhead):
      y += self.att_layers[i](x,padding_mask,enc=enc,mask=mask,other_mask=other_mask)
    y = self.drop(y)
    return y

In [ ]:
class FFN(nn.Module):
  def __init__(self,dmodel,df,dropout=0.1):
    super().__init__()
    self.W1 = nn.Linear(dmodel,df)
    self.W2 = nn.Linear(df,dmodel)
    self.drop = nn.Dropout(p=dropout)
  
  def forward(self,x):
    x = self.W1(x)
    x = F.relu(x)
    x = self.W2(x)
    x = self.drop(x)
    return x

In [ ]:
class Encoder(nn.Module):
  def __init__(self,dmodel,df,nhead):
    super().__init__()
    self.norm = LayerNorm(dmodel)
    self.mha = TPMultiHeadAttention(dmodel,nhead)
    self.norm1 = LayerNorm(dmodel)
    self.ffn = FFN(dmodel,df)
    self.norm2 = LayerNorm(dmodel)
  
  def forward(self,x,padding_mask):
    x = self.norm(x)
    z = self.mha(x,padding_mask)
    z = self.norm1(x+z)
    y = self.ffn(z)
    return self.norm2(z+y)

In [ ]:
class Decoder(nn.Module):
  def __init__(self,dmodel,df,nhead):
    super().__init__()
    self.norm = LayerNorm(dmodel)
    self.masked_mha = TPMultiHeadAttention(dmodel,nhead)
    self.norm1 = LayerNorm(dmodel)
    self.enc_dec_attention = TPMultiHeadAttention(dmodel,nhead)
    self.norm2 = LayerNorm(dmodel)
    self.ffn = FFN(dmodel,df)
    self.norm3 = LayerNorm(dmodel)
  
  def forward(self,x,enc,padding_mask,other_mask):
    x = self.norm(x)
    z1= self.masked_mha(x,padding_mask,mask=True)
    z1 = self.norm1(x+z1)
    z2= self.enc_dec_attention(z1,padding_mask,enc=enc,mask=False,other_mask=other_mask)
    z2 = self.norm2(z1+z2)
    y = self.ffn(z2)
    return self.norm3(z2+y)

In [ ]:
class Transformer(nn.Module):
  def __init__(self,voc_size,dmodel,df,nhead,nlayers,dropout=0.1):
    super().__init__()
    self.nlayers = nlayers
    self.embedding = EmbeddingLayer(voc_size,dmodel)
    self.encoders = nn.ModuleList([Encoder(dmodel,df,nhead) for i in range(nlayers)])
    self.decoders = nn.ModuleList([Decoder(dmodel,df,nhead) for i in range(nlayers)])
    self.drop1 = nn.Dropout(p=dropout)
    self.drop2 = nn.Dropout(p=dropout)

  def forward(self,x,z,in_padding_mask,out_padding_mask,encoding=True):
    if encoding:
      emb = self.embedding(x)
      t = position_embedding(x.shape[0],x.shape[1],dmodel,in_padding_mask)
      x = emb + t
      x = self.drop1(x)
      for i in range(self.nlayers):
        x = self.encoders[i](x,in_padding_mask)

    emb = self.embedding(z)
    t = position_embedding(z.shape[0],z.shape[1],dmodel,out_padding_mask)
    z = emb + t
    z = self.drop2(z)
    for i in range(self.nlayers):
      z = self.decoders[i](z,x,out_padding_mask,in_padding_mask)
    
    z = z @ self.embedding.E.weight.T

    return x,z

In [ ]:
in_sentences = ["<sos> dai ragazzi per una volta che ci andiamo non scegliamo il posto che fa pagare poco <eos>","<sos> altrimenti tanto vale andare a mensa <eos> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>","<sos> importante è la compagnia <eos> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>"]
in_vocabulary = create_vocabulary(in_sentences)

In [ ]:
out_sentences = ["<sos> come on guys for once let's not choose the place that charges little <eos>","<sos> otherwise we might as well go to the canteen <eos> <pad> <pad> <pad> <pad>","<sos> important is company <eos> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>"]
out_vocabulary = create_vocabulary(out_sentences)

In [ ]:
print(in_vocabulary)
print(out_vocabulary)

{'<sos>': 1, '<eos>': 2, '<pad>': 0, 'dai': 3, 'ragazzi': 4, 'per': 5, 'una': 6, 'volta': 7, 'che': 8, 'ci': 9, 'andiamo': 10, 'non': 11, 'scegliamo': 12, 'il': 13, 'posto': 14, 'fa': 15, 'pagare': 16, 'poco': 17, 'altrimenti': 18, 'tanto': 19, 'vale': 20, 'andare': 21, 'a': 22, 'mensa': 23, 'importante': 24, 'è': 25, 'la': 26, 'compagnia': 27}
{'<sos>': 1, '<eos>': 2, '<pad>': 0, 'come': 3, 'on': 4, 'guys': 5, 'for': 6, 'once': 7, "let's": 8, 'not': 9, 'choose': 10, 'the': 11, 'place': 12, 'that': 13, 'charges': 14, 'little': 15, 'otherwise': 16, 'we': 17, 'might': 18, 'as': 19, 'well': 20, 'go': 21, 'to': 22, 'canteen': 23, 'important': 24, 'is': 25, 'company': 26}


In [ ]:
input,in_pad_mask = encoding(in_vocabulary,in_sentences)
teacher,out_pad_mask = encoding(out_vocabulary,out_sentences)

In [ ]:
dmodel = 512
dk,dv = 64,64
nhead = 8
df = 2048
nlayers = 6
in_vocabulary_size = len(in_vocabulary.keys())
out_vocabulary_size = len(out_vocabulary.keys())

In [ ]:
print(in_vocabulary_size)
print(out_vocabulary_size)

28
27


In [ ]:
tran = Transformer(in_vocabulary_size,dmodel,df,nhead,nlayers)

In [ ]:
loss = nn.CrossEntropyLoss()
opt = torch.optim.Adam(tran.parameters(),lr=1e-04, betas=(0.9, 0.98), eps=1e-09)

tran.train()

for i in range(100):
    
  opt.zero_grad()
  x,output = tran(input,teacher,in_pad_mask,out_pad_mask)
  pred = output[:,:-1,:].reshape(output.shape[0]*(output.shape[1] - 1),output.shape[2])
  target = teacher[:,1:].reshape(teacher.shape[0]*(teacher.shape[1] - 1))
  l = loss(pred,target)
  l.backward()
  if i%10 == 0 or i == 99:
    print(l)
  #nn.utils.clip_grad_norm_(tran.parameters(), 0.1)
  opt.step()
  

tensor(89.6699, grad_fn=<NllLossBackward0>)
tensor(1.0374, grad_fn=<NllLossBackward0>)


KeyboardInterrupt: ignored

In [ ]:
output.argmax(-1)[:,:-1]

tensor([[ 1,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15],
        [ 1, 16, 17, 18, 19, 20, 21, 22, 11, 23,  2,  0,  0,  0],
        [ 1, 24, 25, 26,  2,  0,  0,  0,  0,  0,  0,  0,  0,  0]])

In [ ]:
teacher[:,1:]

tensor([[ 3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15,  2],
        [16, 17, 18, 19, 20, 21, 22, 11, 23,  2,  0,  0,  0,  0],
        [24, 25, 26,  2,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0]])

In [ ]:
in_test = ["<sos> ragazzi andiamo a mensa <eos> <pad> <pad>","<sos> il posto fa pagare la compagnia <eos>"]
out_test = ["<sos> guys let's go to the canteen <eos>","<sos> the place charges the company <eos> <pad>"]

In [ ]:
input_test,in_test_pad_mask = encoding(in_vocabulary,in_test)
target_test,out_test_pad_mask = encoding(out_vocabulary,out_test)

In [ ]:
x = input_test
enc = True
trg = target_test.clone()
for i in range(1,target_test.shape[1]):
  x,output = tran(x,trg[:,:i],in_test_pad_mask,out_test_pad_mask[:,:i,:i],encoding = enc)
  trg[:,i] = output.argmax(-1)[:,-1]
  enc = False

In [ ]:
output.argmax(-1)

tensor([[ 1, 18,  1,  1, 18,  1,  1],
        [ 1,  1,  1,  1,  1,  1,  1]])

In [ ]:
target_test[:,1:]

tensor([[ 5,  8, 21, 22, 11, 23,  2],
        [11, 12, 14, 11, 26,  2,  0]])